
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel provincial de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

# Extracción de datos de la API

In [ ]:
import requests
import pandas as pd

In [ ]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [ ]:
base_url_demanda = "https://apidatos.ree.es/es/datos/demanda/evolucion"
base_url_generacion = "https://apidatos.ree.es/es/datos/generacion/estructura-renovables"
headers = {'Accept': 'application/json'}


def obtener_datos(tipo, anio, comunidad_id):
    base_url = base_url_demanda if tipo == 'demanda' else base_url_generacion
    fecha_inicio = f"{anio}-01-01"
    fecha_fin = f"{anio}-12-31"
    params = {'start_date': fecha_inicio,
            'end_date': fecha_fin,
            'time_trunc': 'month',
            'geo_limit': 'ccaa',
            'geo_ids': comunidad_id}
    response = requests.get(base_url, headers=headers, params=params)
    
    # Revisar si la solicitud fue exitosa
    if response.status_code == 200:
        data = response.json()
        valores = data['included'][0]['attributes']['values']
        df = pd.DataFrame(valores)
        df['Tipo'] = tipo
        df['Id_Comunidad']=comunidad_id
        return df
    else:
        print(f"Error {response.status_code} al solicitar {tipo} para {comunidad_id} en el año {anio}")
        return pd.DataFrame()

# Extracción de datos con Web Scrapping

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium import webdriver
from tqdm import tqdm
import os
import re
import zipfile
import shutil

### Datos demográficos

In [25]:
url = "https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990"

chrome_options = webdriver.ChromeOptions()

prefs = {
    "download.default_directory": "C:\\Bravo\\Hackio\\Modulo5\\Labs\\laboratorio-modulo5-leccion01-elt-extraccion\\datos\\DatosDescargados",
    "download.prompt_for_download": False,
    "directory_upgrade": True,
}

chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(5)
sleep(1)
driver.find_element("css selector", "#aceptarCookie").click()
driver.find_element("xpath", '/html/body/div[1]/main/section[2]/div[1]/div[1]/div[1]/ul/li/ul/li[3]/a').click()
driver.find_element("xpath", "/html/body/div[1]/main/div[2]/ul/li[4]/ul/li[1]/a").click()
for n in range(1, 4):
    driver.find_element("xpath", f"//*[@id='tg{n}']/div/fieldset/div[2]/button[1]/i").click()
    sleep(1)

driver.find_element("xpath", "/html/body/div/main/form/ul/li[1]/ul/li[5]/div/fieldset/div[3]/button[2]/i").click()
anios = driver.find_elements("css selector", "#periodo")
cant_anios = anios[0].find_elements(By.TAG_NAME, "option")
sleep(1)
for n in range(2, 5):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="periodo"]/option[{n}]'))).click()
    sleep(1)

# Seleccionar el iframe para descargar csv
driver.find_element("css selector", "#botonConsulSele").click()
sleep(1)
driver.find_element("xpath", "/html/body/div[1]/main/ul/li/div/div/form[2]/button/i").click()
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="thickBoxINEfrm"]')))
driver.switch_to.frame(iframe)
sleep(1)
driver.find_element("xpath", "/html/body/form/ul/li[4]/label").click()
sleep(1)
driver.switch_to.default_content()
sleep(1)
driver.close()


In [ ]:
# Esta es otra forma de hacerlo
edades = driver.find_elements("css selector", "#cri2")
sleep(1)
cant_edades = edades[0].find_elements(By.TAG_NAME, "option")
print(len(cant_edades))
sleep(1)
for n in range(2, len(cant_edades) + 1):
    #elemento = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cri1'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cri1"]/option[{n}]'))).click()
    sleep(1)

extranjeros = driver.find_elements("css selector", "#cri2")
sleep(1)
driver.find_element("xpath", "/html/body/div/main/form/ul/li[1]/ul/li[3]/div/fieldset/div[2]/button[2]/i").click()
cant_extranjeros = extranjeros[0].find_elements(By.TAG_NAME, "option")
sleep(1)
for n in range(2, len(cant_extranjeros) + 1):
    #elemento = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cri1'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cri2"]/option[{n}]'))).click()
    sleep(1)

generos = driver.find_elements("css selector", "#cri3")
sleep(1)
driver.find_element("xpath", "/html/body/div/main/form/ul/li[1]/ul/li[4]/div/fieldset/div[2]/button[2]/i").click()
cant_generos = generos[0].find_elements(By.TAG_NAME, "option")
sleep(1)
for n in range(2, len(cant_generos) + 1):
    #elemento = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cri1'))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cri3"]/option[{n}]'))).click()
    sleep(1)


### Datos económicos

In [8]:
url = "https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429"

chrome_options = webdriver.ChromeOptions()

prefs = {
    "download.default_directory": "C:\\Bravo\\Hackio\\Modulo5\\Labs\\laboratorio-modulo5-leccion01-elt-extraccion\\datos\\DatosDescargados",
    "download.prompt_for_download": False,
    "directory_upgrade": True,
}

chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(5)
sleep(1)
driver.find_element("css selector", "#aceptarCookie").click()
driver.find_element("xpath", '/html/body/div[1]/main/div[2]/ul/li[3]/ul/li[1]/a').click()
driver.find_element("xpath", '//*[@id="tg1"]/div/fieldset/div[2]/button[1]/i').click()
driver.find_element("xpath", '//*[@id="caja_periodo"]/div/fieldset/div[3]/button[2]/i').click()
sleep(0.5)
anios = driver.find_elements("css selector", "#periodo")
cant_anios = anios[0].find_elements(By.TAG_NAME, "option")
for n in range(2, 5):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="periodo"]/option[{n}]'))).click()
    sleep(0.5)

driver.find_element("css selector", "#botonConsulSele").click()
sleep(1)
# Seleccionar el iframe para descargar csv
driver.find_element("xpath", "/html/body/div[1]/main/ul[1]/li/div/div/form[2]/button").click()
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="thickBoxINEfrm"]')))
driver.switch_to.frame(iframe)
sleep(1)
driver.find_element("xpath", "/html/body/form/ul/li[4]/label").click()
sleep(1)
driver.switch_to.default_content()
sleep(1)
driver.close()